In [1]:
import folium
import geemap.eefolium as gmap
import ee
import pandas as pd
import json

# ee.Authenticate()
ee.Initialize()

In [2]:
Map = gmap.Map(control=True)
Map.add_child(folium.LayerControl())

In [3]:
def import_aois(csv_loc, Full_Congo_Pull=False, start_date=None,
                end_date=None, days_duration=90, poly_start=0, poly_limit=None):
    
    if Full_Congo_Pull:
        with open(csv_loc,"r",encoding='utf-8') as jsonfile:
            data = json.load(jsonfile)
            for geometry in data["features"]:
                polygon = geometry["geometry"]["coordinates"][0][0]
                poly_obj = ee.Geometry.Polygon(polygon)
                feature = ee.Feature(poly_obj)
                
        return feature



zhenya_cb_no_grid = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/congo_basin_boundary/Congo_Basin_Boundary_no_islands_v5.geojson"

In [4]:
feature = import_aois(zhenya_cb_no_grid, Full_Congo_Pull=True)

In [5]:
Full_Basin_coords = feature.getInfo()["geometry"]["coordinates"]

In [6]:
AOI_full_basin = ee.Geometry.Polygon(Full_Basin_coords)
START_DATE = '2019-01-01'
END_DATE = '2020-12-21'
CLOUD_FILTER = 100


vis = {'bands': 'array'}



def make_s2_sr_col(aoi=AOI_full_basin,start_date = START_DATE, end_date=END_DATE, CLOUD_FILTER=CLOUD_FILTER):
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))
    return s2_sr_col

def visualize_geo(coords,Map = Map):
    Map = gmap.Map()
    geo_obj = ee.Geometry.Polygon(coords)
    Map.centerObject(geo_obj,3)
    Map.add_layer(geo_obj, {}, 'default display')
    Map.add_child(folium.LayerControl())
    return Map

def visualize_raster(img):
    Map = gmap.Map()
    Map.centerObject(img,10)
    Map.addLayer(img, vis)
    Map.add_child(folium.LayerControl())
    return Map
    

In [7]:
col = make_s2_sr_col()

In [8]:
col.size().getInfo()

69205

In [9]:
col.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32735',
   'crs_transform': [60, 0, 199980, 0, -60, 9500020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32735',
   'crs_transform': [10, 0, 199980, 0, -10, 9500020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32735',
   'crs_transform': [10, 0, 199980, 0, -10, 9500020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32735',
   'crs_transform': [10, 0, 199980, 0, -10, 9500020]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min':

In [44]:
col2 = make_s2_sr_col(CLOUD_FILTER=.05)

In [45]:
col2.size().getInfo()

1178

# Feature or Image Area Calculation

In [31]:
def get_area(feature):
    stateArea = feature.geometry().area()
    stateAreaSqKm = ee.Number(stateArea).divide(1e6).round()
    print(stateAreaSqKm.getInfo(),"square kilometers")
    

get_area(feature)

2508463 square kilometers


# Image Area Calculation

In [37]:
get_feature_area(col.first())

3600 square kilometers


In [46]:
# ee.Image(col.get(2)).getInfo()

listOfImages = col2.toList(col.size())

In [49]:
for i in range(0,20):
    test_image = ee.Image(listOfImages.get(i))
    get_area(test_image)

114 square kilometers
561 square kilometers
221 square kilometers
4761 square kilometers
2075 square kilometers
12081 square kilometers
12100 square kilometers
11463 square kilometers
1 square kilometers
7 square kilometers
8331 square kilometers
544 square kilometers
7593 square kilometers
12112 square kilometers
12110 square kilometers
12118 square kilometers
196 square kilometers
2140 square kilometers
5145 square kilometers
2799 square kilometers


# Composite Metrics

In [ ]:
cloudiness_list = col.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo()

In [11]:
sum(cloudiness_list) / len(cloudiness_list)

63.74348329882404

In [14]:
df = pd.DataFrame(cloudiness_list)

In [15]:
df.value_counts()

100.000000    3268
99.999999      686
99.999997      583
99.999998      472
0.000000       261
              ... 
42.414691        1
42.415445        1
42.415534        1
42.416303        1
67.783577        1
Length: 62349, dtype: int64

In [16]:
df

,0
0,97.747635
1,90.111278
2,99.429986
3,96.774037
4,77.805116
...,...
69200,11.530281
69201,7.609377
69202,0.124518
69203,0.019859


# 4% of Basin is Relatively CloudFree Over a Two Year Period

In [53]:
df[df[0] < 1].shape[0] / df.shape[0]

0.038219781807672853

# .4% of Basin is Completely CloudFree Over a Two Year Period

In [54]:
df[df[0] == 0].shape[0] / df.shape[0]

0.0037714038003034463

# 35% of CB less than %50 cloudy Over Two Year Period

In [57]:
df[df[0] <= 50].shape[0] / df.shape[0]

0.3511595982949209